<a href="https://colab.research.google.com/github/threegenie/similarity_search/blob/main/0404_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> pandas, numpy import

In [3]:
import pandas as pd
import numpy as np
import re
import time

> data import

In [4]:
data = pd.read_csv('/content/drive/My Drive/0404/text_classification.csv')

In [5]:
data.head()

,text,category
0,이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림...,연예/문화
1,"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요...",연예/문화
2,인천시 서구와 계양구에서 경기도 부천을 거쳐 서울 홍대까지 지하철이 건설될 것으로 ...,생활
3,김성태 자유한국당 의원이 최근 황운하 울산경찰청장과 벌인 ‘미친개’ 공방을 언급하며...,정치
4,"""지난해 사상 최대 실적을 거뒀던 우리 수출이 새해에도 좋은 흐름을 이어갔습니다. ...",스포츠


In [6]:
df = data[:5000] #임의로 5천건만 실험

In [7]:
del data

> 전처리

In [8]:
df['text'] = [re.sub('[^A-Za-z0-9가-힣 ]', '', s) for s in df['text']]
df.replace('', np.nan, inplace=True) 

> mecab 설치

In [9]:
# %%capture
# %%bash
# apt-get update
# apt-get install g++ openjdk-8-jdk python-dev python3-dev
# pip install --upgrade pip
# pip install JPype1-0.5.7-cp27-none-win_amd64.whl
# # pip3 install JPype1
# pip3 install konlpy

In [10]:
# %env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [11]:
# %%capture
# %%bash
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# pip3 install /tmp/mecab-python-0.996

갑자기 잘 되던 mecab 설치가 오류나서... 귀찮은 마음에 komoran으로 대체... 다음에 필요할 땐 mecab 쓰자 ㅎㅎ...

In [14]:
%%capture
!pip install konlpy

In [17]:
import konlpy
# from konlpy.tag import Mecab
from konlpy.tag import Komoran

> tokenize 적용

In [15]:
# mecab = Mecab()

In [18]:
komoran = Komoran()

In [19]:
stopwords = ['이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

In [26]:
def tokenizer(raw, pos=["NNG","NNP"]):
#def tokenizer(raw, pos=["NNG","NNP","VV","VA"],stopwords=stopwords):
  komoran = Komoran()
  return [word for word, tag in komoran.pos(raw) if len(word)>1 and tag in pos]

In [27]:
# df['tokenized'] = df['text'].apply(mecab.nouns)

In [28]:
# df['tokenized'] = [','.join(map(str,i)) for i in df['tokenized']]

> TfidVectorizer + KNN

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tf = TfidfVectorizer(tokenizer=tokenizer,min_df=3,analyzer='word',max_features=10)

In [ ]:
dtf = tf.fit_transform(df['text'])

In [ ]:
dtf.shape

In [ ]:
dtf = pd.DataFrame(dtf.todense(), columns=tf.get_feature_names())

> cosine 유사도

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similar = cosine_similarity(dtf,dtf)

In [ ]:
dm = pd.DataFrame(cosine_similar)

In [ ]:
df['idxmax'] = dm[dm < (1-1e-6)].idxmax()

In [ ]:
df['max'] = dm[dm < (1-1e-6)].max()

In [ ]:
result = (df['max']>0.9)

In [ ]:
result

In [ ]:
result.to_csv('/content/drive/My Drive/0404/result.csv',index=False)